In [115]:
import os
#import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from download_imdb import maybe_download_imdb
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.metrics import classification_report, confusion_matrix, precision_recall_fscore_support
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer

In [2]:
stop_string = """
's
a
able
about
above
abst
accordance
according
accordingly
across
act
actually
added
adj
affected
affecting
affects
after
afterwards
again
against
ah
all
almost
alone
along
already
also
although
always
am
among
amongst
an
and
announce
another
any
anybody
anyhow
anymore
anyone
anything
anyway
anyways
anywhere
apparently
approximately
are
aren
arent
arise
around
as
aside
ask
asking
at
auth
available
away
awfully
back
be
became
because
become
becomes
becoming
been
before
beforehand
begin
beginning
beginnings
begins
behind
being
believe
below
beside
besides
between
beyond
biol
both
brief
briefly
but
by
came
can
cannot
can't
cause
causes
certain
certainly
co
com
come
comes
contain
containing
contains
could
couldnt
date
did
didn't
different
do
does
doesn't
doing
done
don't
down
downwards
due
during
each
ed
edu
effect
eg
eight
eighty
either
else
elsewhere
end
ending
enough
especially
et
et-al
etc
even
ever
every
everybody
everyone
everything
everywhere
ex
except
far
few
fifth
first
five
fix
followed
following
follows
for
former
formerly
forth
found
four
from
further
furthermore
gave
get
gets
getting
give
given
gives
giving
go
goes
gone
got
gotten
had
happens
hardly
has
hasn't
have
haven't
having
he
hed
hence
her
here
hereafter
hereby
herein
heres
hereupon
hers
herself
hes
hi
hid
him
himself
his
hither
home
how
howbeit
however
hundred
i
id
ie
if
i'll
im
immediate
immediately
importance
important
in
inc
indeed
index
information
instead
into
invention
inward
is
isn't
it
itd
it'll
its
itself
i've
just
keep
keeps
kept
kg
km
know
known
knows
largely
last
lately
later
latter
latterly
least
less
lest
let
lets
like
liked
likely
line
little
'll
look
looking
looks
ltd
made
mainly
make
makes
many
may
maybe
me
mean
means
meantime
meanwhile
merely
mg
might
million
miss
ml
more
moreover
most
mostly
mr
mrs
much
mug
must
my
myself
name
namely
nay
near
nearly
necessarily
necessary
need
needs
neither
never
nevertheless
new
next
nine
ninety
no
nobody
non
none
nonetheless
noone
nor
normally
nos
not
n't
noted
nothing
now
nowhere
obtain
obtained
obviously
of
off
often
oh
ok
okay
old
omitted
on
once
one
ones
only
onto
or
ord
other
others
otherwise
ought
our
ours
ourselves
out
outside
over
overall
owing
own
p
page
pages
part
particular
particularly
past
per
perhaps
placed
please
plus
poorly
possible
possibly
potentially
predominantly
present
previously
primarily
probably
promptly
proud
provides
put
que
quickly
quite
ran
rather
readily
really
recent
recently
ref
refs
regarding
regardless
regards
related
relatively
research
respectively
resulted
resulting
results
right
run
said
same
saw
say
saying
says
sec
section
see
seeing
seem
seemed
seeming
seems
seen
self
selves
sent
seven
several
shall
she
shed
she'll
shes
should
shouldn't
show
showed
shown
showns
shows
significant
significantly
similar
similarly
since
six
slightly
so
some
somebody
somehow
someone
somethan
something
sometime
sometimes
somewhat
somewhere
soon
sorry
specifically
specified
specify
specifying
still
stop
strongly
sub
substantially
successfully
such
sufficiently
suggest
sup
sure
"""

In [3]:
stops = set(stop_string.strip().splitlines())

In [78]:
def clean_y(y_raw):
    """
    Strips out True and False values from the dicts in y_raw. Makes and returns an np array of dtype=int from the values
    """
    y_clean = np.array([item["POSITIVE"] for item in y_raw], dtype=int)
    return y_clean


filename = 'imdb_thinc_data.pickle'
filepath = os.path.join('..','data','thinc',filename)
X_train_raw, y_train_raw , X_test_raw, y_test_raw =maybe_download_imdb(filepath)


In [89]:
#TTD convert X_train_raw and X_test_raw into a list in maybe_download_imdb
X_train_raw = list(X_train_raw)
X_test_raw = list(X_test_raw)

In [90]:
len(X_train_raw), len(y_train_raw), len(X_test_raw), len(y_test_raw)

(20000, 20000, 5000, 5000)

In [91]:
X_train_raw[42], y_train_raw[42]

('Unlike other commenters who have commented on this movie\'s ability to transcend race, contrarily, I think that this powerful film provides a complex and deep story that addresses institutional racism and the effects thereof. Washington directs Fisher\'s story with a careful hand and critical eye, relinquishing this cinematic endeavor neither to dismemberment of women\'s bodies, perpetuating unthoughtful stereotypes, nor satisfying the expectation of the white gaze. I think this film might be a bit too happy in the end; however, it is deeply entrenched in Afro-American culture and discourse to the point that some white spectators may get the feeling of looking into the life of this Afro-American--Antwone Fisher. I have problems with the Naval aspect of the film, but when we look at America, there are not many choices or opportunities for black men who are/were in Fisher\'s situation or similar situations. Viewers may go to this movie expecting a "Black Movie: what is a "Black Movie?"

In [92]:
type(X_train_raw), type(y_train_raw),type(X_train_raw[42]),  type(y_train_raw[42])

(list, list, str, dict)

In [93]:
type(X_test_raw), type(y_test_raw),type(X_test_raw[42]),  type(y_test_raw[42])

(list, list, str, dict)

In [94]:
#use this as supplied tokenizer
class LemmaTokenizer(object):
    def __init__(self):
        self.wnl = WordNetLemmatizer()
    def __call__(self, doc):
        return [self.wnl.lemmatize(t) for t in word_tokenize(doc)]

In [95]:
vec = CountVectorizer(ngram_range=(1, 1), 
                      stop_words=stops, 
                      strip_accents='unicode',
                     token_pattern=r'\b[a-zA-Z]{3,}\b',
                     tokenizer=LemmaTokenizer())

counts = vec.fit_transform(X_train_raw)
transformer = TfidfTransformer()
X_train_tfidf = transformer.fit_transform(counts)
y_train = clean_y(y_train_raw)

In [96]:
len(vec.get_feature_names())

87049

In [118]:
#ran param_grid={'C':[1e-3,1e-2,1e-1,1,2,5,10,100,1000,10000]} ===> 10
#ran param_grid= {'C':[5,10,15,20,30,40,50,75,100]}===> 10


param_grid= {'C':[5,10,15,20,30,40,50,75,100]}


In [121]:
clf = LogisticRegression(random_state=42)
c_validator = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

In [125]:
grid = GridSearchCV(clf, n_jobs=-1, param_grid=param_grid,cv=c_validator, scoring='f1')
grid

GridSearchCV(cv=StratifiedKFold(n_splits=10, random_state=42, shuffle=True),
       error_score='raise',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=42, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'C': [5, 10, 15, 20, 30, 40, 50, 75, 100]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='f1', verbose=0)

In [126]:
grid.fit(X_train_tfidf,y_train)

GridSearchCV(cv=StratifiedKFold(n_splits=10, random_state=42, shuffle=True),
       error_score='raise',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=42, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'C': [5, 10, 15, 20, 30, 40, 50, 75, 100]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='f1', verbose=0)

In [127]:
clf = grid.best_estimator_

In [128]:
clf.get_params

<bound method BaseEstimator.get_params of LogisticRegression(C=10, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=42, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)>

In [129]:
grid.best_score_

0.89312404202985418

In [130]:
counts_test = vec.transform(X_test_raw)

X_test_tfidf = transformer.transform(counts_test)
y_test = clean_y(y_test_raw)

In [134]:
y_pred_test = clf.predict(X_test_tfidf)

In [135]:
confusion_matrix(y_pred_test, y_test)

array([[2211,  247],
       [ 294, 2248]])

In [136]:
precision_recall_fscore_support(y_pred_test, y_test)

(array([ 0.88263473,  0.901002  ]),
 array([ 0.8995118 ,  0.88434304]),
 array([ 0.89099335,  0.8925948 ]),
 array([2458, 2542]))

In [137]:
# save model
from sklearn.externals import joblib
joblib.dump(clf, 'best_log_reg.pkl') 

['best_log_reg.pkl']

In [74]:
logreg.score(X_train_tfidf,y_array)

1.0

In [75]:
logreg.get_params()

{'C': 20,
 'class_weight': None,
 'dual': False,
 'fit_intercept': True,
 'intercept_scaling': 1,
 'max_iter': 100,
 'multi_class': 'ovr',
 'n_jobs': 1,
 'penalty': 'l2',
 'random_state': 42,
 'solver': 'liblinear',
 'tol': 0.0001,
 'verbose': 0,
 'warm_start': False}

In [76]:
y_pred = logreg.predict(X_tfidf)

In [77]:
confusion_matrix(y_array, y_pred)

array([[27,  0],
       [ 0, 23]])